# Modulation format demo

This demo shows constellation diagrams of *M*-PSK and *M*-QAM for different modulation orders, and the effects of signal-to-noise-ratio (SNR) on the bit error rate (BER). Use the sliders to change the modulation format and SNR. Compare the BER for different modulation formats at the same SNR. 

* How does the BER differ between different between different modulation order *M*?
* What is the difference between PSK and QAM modulation?
    * Do they have the same BER for the same *M*?
    * Does this change with *M*?
    
*Note that for performance reasons this only simulates 40,000 symbols, so BERs below $10^{-4}$ are increasingly inaccurate and will flactuate for different runs, because there are not enough errors.*

In [9]:
#%pylab inline
from qampy import signals, impairments
import numpy as np
import matplotlib.pylab as plt
from ipywidgets import widgets
from IPython.display import display
from ipywidgets.widgets.interaction import interact, interact_manual

In [10]:
def cal_psk_symbols(M):
    syms = np.zeros(M, dtype=np.complex128)
    for i in range(M):
        syms[i] = np.exp(-2j*np.pi*(i/M+3/(2*M)))#/np.sqrt(2)
    return syms

In [11]:
class SignalPSKGrayCoded(signals.SignalQAMGrayCoded):
    @classmethod
    def _generate_mapping(cls, M, scale, dtype=np.complex128):
        Nbits = np.log2(M)
        symbols = cal_psk_symbols(M)
        # check if this gives the correct mapping
        #symbols /= scale
        _graycode = signals.theory.bin2gray(np.arange(M))
        coded_symbols = symbols[_graycode]
        bformat = "0%db" % Nbits
        encoding = dict([(symbols[i],
                          signals.bitarray(format(_graycode[i], bformat)))
                         for i in range(len(_graycode))])
        bitmap_mtx = signals.generate_bitmapping_mtx(coded_symbols, cls._demodulate(coded_symbols, encoding), M, dtype=dtype)
        return coded_symbols, _graycode, encoding, bitmap_mtx


In [12]:
def plot_constellation(M=4, constype="QAM", snr=100):
    constype = constype.lower()
    if M > 2**12:
        M = 2**12
    if constype == "psk":
        sig = signals.SignalPSKGrayCoded(M, 4*10**4, nmodes=1)
    else:
        sig = signals.SignalQAMGrayCoded(M, 4*10**4, nmodes=1)
    
    sig = impairments.change_snr(sig, snr)
    fig = plt.figure(figsize=(8,8))
    ax = plt.subplot(111)
    ber = np.mean(sig.cal_ber())
    bers, o = "{:e}".format(ber).split("e")
    if snr > 40:
        plt.plot(sig[0].real, sig[0].imag, '.')
        ax.text(0.7, 0.05, r"BER = ${:.1f}\cdot 10^{{{:d}}}$".format(float(bers), int(o)), color="black", transform=ax.transAxes)
    else:
        plt.hist2d(sig[0].real, sig[0].imag, bins=200, range=np.array([[-1.4, 1.4], [-1.4, 1.4]]))
        ax.text(0.7, 0.05, r"BER = ${:.1f}\cdot 10^{{{:d}}}$".format(float(bers), int(o)), color="white", transform=ax.transAxes)
    ax.set_aspect("equal")
    ax.set_xlim([-1.4, 1.4])
    ax.set_ylim([-1.4, 1.4])
    #plt.show()

In [13]:
def plot_constellation_interactive():
    #M =  widgets.Sliderlog(description="M", min=2, max=12, valinit=4, valfmt='%d', closedmin=True, closedmax=True, slidermin=None, slidermax=None, dragging=True, valstep=1, orientation='horizontal')
    M = widgets.SelectionSlider(description="M", options=[2**i for i in range(1, 13)], value=4)
    constype = widgets.RadioButtons(description="Constellation", options=["QAM", "PSK"], value="QAM")
    snr = widgets.FloatSlider(description="SNR", min=-5, max=60, value=20, continuous_update=False)#, valfmt="%.1f dB")
    interact(plot_constellation, constype=constype, snr=snr, M=M)

In [14]:
plot_constellation_interactive()

interactive(children=(SelectionSlider(description='M', index=1, options=(2, 4, 8, 16, 32, 64, 128, 256, 512, 1…

In [15]:
def plot_scan(fmts=["4-QAM"]):
    snrs = np.linspace(0, 35, 35)
    plt.figure(figsize=(12,8))
    for i, fmt in enumerate(fmts):
        M, ct = fmt.split('-')
        M = int(M)
        bers = np.zeros_like(snrs)
        if ct == "PSK":
            sig = signals.SignalPSKGrayCoded(M, 4*10**4, nmodes=1)
        else:
            sig = signals.SignalQAMGrayCoded(M, 4*10**4, nmodes=1)
        for i, snr in enumerate(snrs):
            s2 = impairments.change_snr(sig, snr)
            bers[i] = np.mean(s2.cal_ber())
        plt.semilogy(snrs, bers, label=fmt)
    plt.xlabel("SNR [dB]")
    plt.ylabel("BER")
    plt.xlim([0,35])
    plt.legend()
    

In [16]:
def plot_scan_interactive():
    fmts = widgets.SelectMultiple(description="Formats", value=("4-QAM",), options=["4-PSK", "4-QAM", "16-PSK", "16-QAM", "32-PSK", "32-QAM", "64-PSK", "64-QAM"], continuous_update=False)
    interact_manual(plot_scan, fmts=fmts)

## Compare BERs

In the interact below you can compare the BER performance of different modulation formats. Select the formats you want to compare and press *"Run Interact"*. This might take a moment. 

In [17]:
plot_scan_interactive()

interactive(children=(SelectMultiple(description='Formats', index=(1,), options=('4-PSK', '4-QAM', '16-PSK', '…